# Convolutional Neural Network Implementation




---

## Implementation by Subclassing `tf.keras.models.Model`

Subclassing `Model` (instead of using `Sequential`) gives you:

- Full control over the forward pass (call)

- Explicit handling of training vs inference

- Easier extension to more complex architectures

This is the recommended approach for anything non-trivial in terms of neural network architecture.

In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

tf.random.set_seed(69)

(x_train, y_train), (x_test, y_test) = mnist.load_data()

# MNIST data comes as (N, 28, 28); we need to add the channel dimension for CNN to work
x_train = x_train[..., tf.newaxis]  # (N, 28, 28, 1)
x_test = x_test[..., tf.newaxis]  # (N, 28, 28, 1)

# normalization
x_train = (x_train/ 255.0).astype("float32")
x_test = (x_test/ 255.0).astype("float32")

/Users/pjc/Downloads/cnn-implementation/.venv/lib/python3.11/site-packages/keras/src/export/tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


In [ ]:
# ---------------------
#  model architecture
# ---------------------

class ConvNet(Model):
    def __init__(self):
        super().__init__()

        # first convolutional block - 3x3x32 filter + relu + 2x2 maxpool
        self.conv1 = Conv2D(
            filters=32,
            kernel_size=3,
            padding="same",
            activation="relu",
            name="Conv1"
        )
        self.pool1 = MaxPooling2D(pool_size=2, strides=2)

        # second convolutional block - 3x3x64 filter + relu + 2x2 maxpool
        self.conv2 = Conv2D(
            filters=64,
            kernel_size=3,
            padding="same",
            activation="relu",
            name="Conv2"
        )
        self.pool2 = MaxPooling2D(pool_size=2, strides=2)

        # fully connected layers - 128 neurons + relu + dropout + 10 neurons (logits)
        self.flatten = Flatten()
        self.fc1 = Dense(128, activation="relu", name="FC1")
        self.dropout = Dropout(0.5, name="Dropout") # dropout layer for regularization
        self.fc2 = Dense(10, name="Logits_Output")  # logits are the outputs
        

    def call(self, x, training=False): # invoked during training aswell as inference
        # 28x28x1 -> 14x14x32
        x = self.pool1(self.conv1(x))

        # 14x14x32 -> 7x7x64
        x = self.pool2(self.conv2(x))

        # 7x7x64 -> 3136
        x = self.flatten(x)

        # fully connected layers
        x = self.fc1(x)
        x = self.dropout(x, training=training)
        x = self.fc2(x)

        return x

In [ ]:
# --------------------------------
# setting up and training the model
# ---------------------------------

model = ConvNet()

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

model.compile(
    optimizer=optimizer,
    loss=loss_fn,
    metrics=["accuracy"]
)

history=model.fit(
    x_train,y_train,
    epochs=5,
    batch_size=512,
    validation_split=0.2
)

training_loss, training_accuracy =model.evaluate(x_train, y_train)
test_loss, test_accuracy =model.evaluate(x_test, y_test)

Epoch 1/5
94/94 ━━━━━━━━━━━━━━━━━━━━ 7s 73ms/step - accuracy: 0.8049 - loss: 0.6261 - val_accuracy: 0.9601 - val_loss: 0.1331
Epoch 2/5
94/94 ━━━━━━━━━━━━━━━━━━━━ 7s 73ms/step - accuracy: 0.9524 - loss: 0.1632 - val_accuracy: 0.9777 - val_loss: 0.0740
Epoch 3/5
94/94 ━━━━━━━━━━━━━━━━━━━━ 7s 74ms/step - accuracy: 0.9678 - loss: 0.1112 - val_accuracy: 0.9823 - val_loss: 0.0610
Epoch 4/5
94/94 ━━━━━━━━━━━━━━━━━━━━ 7s 73ms/step - accuracy: 0.9745 - loss: 0.0879 - val_accuracy: 0.9856 - val_loss: 0.0496
Epoch 5/5
94/94 ━━━━━━━━━━━━━━━━━━━━ 7s 74ms/step - accuracy: 0.9785 - loss: 0.0722 - val_accuracy: 0.9876 - val_loss: 0.0445
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9896 - loss: 0.0355
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9881 - loss: 0.0351


In [3]:
model.summary()

Model: "conv_net"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ Conv1 (Conv2D)                  │ (None, 28, 28, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Conv2 (Conv2D)                  │ (None, 14, 14, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 3136)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ FC1 (Dense)                     │ (None, 128)            │       401,536 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Logits_Output (Dense)           │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,264,928 (4.83 MB)

 Trainable params: 421,642 (1.61 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 843,286 (3.22 MB)

## Implementation using Functional APIs

We can implement the same using Functional API of keras. Here's how:

In [67]:
import tensorflow as tf
from tensorflow.keras.layers import (
    Input, Conv2D, MaxPooling2D,
    Flatten, Dense, Dropout
)
from tensorflow.keras.models import Model

inputs = Input(shape=(28, 28, 1), name="Input")

# First convolutional block
x = Conv2D(
    filters=32,
    kernel_size=3,
    padding="same",
    activation="relu",
    name="Conv1"
)(inputs)
x = MaxPooling2D(pool_size=2, strides=2, name="Pool1")(x)

# Second convolutional block
x = Conv2D(
    filters=64,
    kernel_size=3,
    padding="same",
    activation="relu",
    name="Conv2"
)(x)
x = MaxPooling2D(pool_size=2, strides=2, name="Pool2")(x)

# Fully connected layers
x = Flatten(name="Flatten")(x)
x = Dense(128, activation="relu", name="FC1")(x)
x = Dropout(0.5, name="Dropout")(x)

# Output layer — logits
outputs = Dense(10, name="Logits")(x)

model = Model(inputs=inputs, outputs=outputs, name="ConvNet")

model.compile(
    optimizer="adam",
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"]
)

model.summary()

Model: "ConvNet"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ Input (InputLayer)              │ (None, 28, 28, 1)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Conv1 (Conv2D)                  │ (None, 28, 28, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Pool1 (MaxPooling2D)            │ (None, 14, 14, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Conv2 (Conv2D)                  │ (None, 14, 14, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Pool2 (MaxPooling2D)            │ (None, 7, 7, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Flatten (Flatten)               │ (None, 3136)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ FC1 (Dense)                     │ (None, 128)            │       401,536 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Logits (Dense)                  │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 421,642 (1.61 MB)

 Trainable params: 421,642 (1.61 MB)

 Non-trainable params: 0 (0.00 B)

In [68]:
model.fit(x_train, y_train, epochs=5, batch_size=512, validation_split=0.2)

Epoch 1/5
94/94 ━━━━━━━━━━━━━━━━━━━━ 8s 83ms/step - accuracy: 0.8135 - loss: 0.6164 - val_accuracy: 0.9594 - val_loss: 0.1334
Epoch 2/5
94/94 ━━━━━━━━━━━━━━━━━━━━ 7s 77ms/step - accuracy: 0.9506 - loss: 0.1675 - val_accuracy: 0.9771 - val_loss: 0.0791
Epoch 3/5
94/94 ━━━━━━━━━━━━━━━━━━━━ 7s 74ms/step - accuracy: 0.9666 - loss: 0.1169 - val_accuracy: 0.9812 - val_loss: 0.0631
Epoch 4/5
94/94 ━━━━━━━━━━━━━━━━━━━━ 7s 77ms/step - accuracy: 0.9725 - loss: 0.0927 - val_accuracy: 0.9844 - val_loss: 0.0524
Epoch 5/5
94/94 ━━━━━━━━━━━━━━━━━━━━ 7s 79ms/step - accuracy: 0.9757 - loss: 0.0795 - val_accuracy: 0.9852 - val_loss: 0.0496


In [69]:
model.evaluate(x_train, y_train), model.evaluate(x_test, y_test)

1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.9872 - loss: 0.0416
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9859 - loss: 0.0402


([0.041598498821258545, 0.9872166514396667],
 [0.04024488106369972, 0.9858999848365784])